In [1]:
import os
import sys
import numpy as np
import torch
from torch import nn
from transformers import pytorch_utils as torch_utils
from peft import LoraConfig

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'mps'

In [3]:
import importlib
import src.train
import src.model

importlib.reload(src.train)
importlib.reload(src.model)

from src.train import sft_train_lora
from src.model import identify_target_modules
from data.gsm8k import GSM8K

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

In [5]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [6]:
from huggingface_hub import login
login()

In [10]:
# dataset = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k", split="train")


model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

dataset = GSM8K(tokenizer=tokenizer)

In [11]:
dataset[50]

{'input_ids': tensor([    2,   534, 26755, 12500,    68,  1866,    10,   353,    15,  3403,
          4217,     4,   832,   191,    16,   204,   377,   251,     4,    91,
          1072,     7,   304,     5,   377,    37,    18,    45,   816,  3403,
             7,  1871,    62,    30,   910,  7520,     6, 23439,   154,     6,
             8,   475,  6932, 11853,    29,     4,    91,  1103,    68,   698,
            13,   349,     4,  1336,   171, 32824,   473,    37,   240,     7,
           674,    10,   353,     7,  1871,    62,    13,    39,  4217,   116,
         22560, 31652,    35,    91,   782,     7,  1871,    62,    68,  4017,
           142,   204,  3023,   727,  5457, 48188,   306,  3226,  1866,  5214,
          4017, 44226,  4017, 50118,   894,    34,   290,   377,     7,  4073,
            42,   418,   142,   316,   111,   204,  5457, 48188,  1092,    12,
           306,  5214,   398, 44226,   398, 50118,   894,   782,     7,  4073,
            68,  1096,    10,   353,   

In [12]:
len(dataset)

7473

In [13]:
target_modules = identify_target_modules(model, name_segment='self_attn')
print(target_modules)

In [14]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
)

In [15]:
sft_train_lora(
    base_model=model,
    train_dataset=dataset,
    eval_dataset=dataset,
    tokenizer=AutoTokenizer.from_pretrained("facebook/opt-350m"),
    adapter_name="sft_lora",
    response_template=" ### Answer:",
    lora_config=lora_config,
)

/opt/anaconda3/envs/reasoning-decomp/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


  0%|          | 0/2805 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 9.05 GB, other allocations: 12.70 MB, max allowed: 9.07 GB). Tried to allocate 11.31 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).